In [1]:

from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()


from azure.ai.ml import MLClient
ml_client = MLClient(
    credential=credential,
    subscription_id="8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5",
    resource_group_name="sparc2023-workspace-xudyu-rg",
    workspace_name="sparc2023-ws-xudyu",
)




In [2]:
from azure.ai.ml.entities import Environment
import os
project_dir="."
dependencies_dir = os.path.join(project_dir,"dependencies/")

pipeline_job_env = Environment(
    name="conda-6DIM",
    description="env for 6DIMCOCO experiments",
    conda_file=os.path.join(dependencies_dir, "conda.yml"),
    image="mcr.microsoft.com/azureml/curated/acpt-pytorch-2.0-cuda11.7:15"
    #set OS var
    
    )

env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name conda-6DIM is registered to workspace, the environment version is 39


In [3]:
from azure.ai.ml import command
from azure.ai.ml import Input,Output
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes


'''        #more info at https://williamfalcon.github.io/test-tube/hyperparameter_optimization/HyperOptArgumentParser/
        self.add_argument("--dir",default="/nobackup/projects/bdlan05/smander3/data",type=str)
        self.add_argument("--log_path",default="/nobackup/projects/bdlan05/smander3/logs/",type=str)
        '''

outputs = {
    "log": Output(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/",
                        mode=InputOutputModes.RW_MOUNT
                        ),
    "annot": Output(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/annotations/",
                        mode=InputOutputModes.RW_MOUNT
                        ),
    "DataNew": Output(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/v2/",
                        mode=InputOutputModes.UPLOAD
                        ),
}


inputs={ "input_data": Input(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/",
                        mode=InputOutputModes.DIRECT,
                        ),
        "annotations": Input(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/annotations/",
                        mode=InputOutputModes.DIRECT,
                        ),
}



registered_model_name = "Contrastive trained DETR Model"
job = command(
    code=project_dir,  # location of source code
    command="python launch.py --num_trials 0 --dir ${{inputs.input_data}} --log_path ${{outputs.log}} --annotations ${{outputs.annot}}",
    environment=pipeline_job_env.name+":"+pipeline_job_env.version,
    compute="smander",
    experiment_name="6DIMCOCO",
    display_name="ContrastiveTraining-6D-StephenM-monothread-pathjoin",
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,
        "node_count": 1,
        "instance_count": 1,
    },
    outputs=outputs,
    inputs=inputs,
)



ml_client.create_or_update(job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


FileNotFoundError: [Errno 2] No such file or directory: '/data/6DIMCOCO/../../blobs/a2a88b96561196777ca173b15309ea859f4d2ce0'

In [22]:
#Lets define some sweeps, We really want to trial a load of values for all the following:
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input, Output
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy
from azure.identity import DefaultAzureCredential
from azure.ai.ml import command
from azure.ai.ml import Input,Output
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes


'''        #more info at https://williamfalcon.github.io/test-tube/hyperparameter_optimization/HyperOptArgumentParser/
        self.add_argument("--dir",default="/nobackup/projects/bdlan05/smander3/data",type=str)
        self.add_argument("--log_path",default="/nobackup/projects/bdlan05/smander3/logs/",type=str)
        '''

outputs = {
    "log": Output(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/",
                        mode=InputOutputModes.RW_MOUNT
                        ),
    "annot": Output(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/annotations/",
                        mode=InputOutputModes.RW_MOUNT
                        ),
    "DataNew": Output(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/v2/",
                        mode=InputOutputModes.UPLOAD
                        ),
                        }


# Create your command
command_job_for_sweep = command(
    code=project_dir,  # location of source code
    command="python launch.py --num_trials 0 --dir ${{inputs.input_data}} --log_path ${{outputs.log}} --annotations ${{outputs.annot}}",#--data ${{inputs.datadir}}",
    environment=pipeline_job_env.name+"@latest",
    compute="cpu-cluster",
    experiment_name="6DIMCOCOWSweep",
    display_name="BigSweep",
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,
    },
    outputs=outputs,
    inputs={
        "input_data": Input(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/",
                        mode=InputOutputModes.DIRECT,
                        ),
        "annotations": Input(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/annotations/",
                        mode=InputOutputModes.DIRECT,
                        ),
        "prune":Choice([1,0]),
        "projection":Choice(["None","inv","iinv"]),
        "normlogits":Choice([1,0]),
        "exactlabels":Choice([1,0]),
        "meanloss":Choice([1,0]),
        "maskLosses":Choice([0,1,2]),
        "logitsversion":Choice([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]),
        "precision":Choice([32,16]),
        "codeversion":Choice([6]),
        "transformer_layers":Choice([3,4,5,6,8,12]),
        "transformer_heads":Choice([16]),
        "embed_dim":Choice([64,128,512]),
        "transformer_width":Choice([64,128,512]),
        "devices":Choice([4]),
        "accelerator":Choice(["gpu"]),
        #"log_path":os.path.join(project_dir,"./logs"),
        "batch_size":Choice(list(range(2,8))),
        "dims":Choice([3,6])
    },
)

# Call sweep() on your command job to sweep over your parameter expressions
sweep_job = command_job_for_sweep.sweep(
    compute="smander", 
    sampling_algorithm="random",
    primary_metric="train_loss",#should really set this to something at the validation stage
    goal="Minimize",
)
# Define the limits for this sweep
sweep_job.set_limits(max_total_trials=500, max_concurrent_trials=20)

# Set early stopping on this one
sweep_job.early_termination = MedianStoppingPolicy(delay_evaluation=5, evaluation_interval=2)

# Specify your experiment details
sweep_job.display_name = "CLIPSWEEP"
sweep_job.experiment_name = "StephenMCLIPHighDimSweep"
sweep_job.description = "Runahyperparametersweep6or3Drepo"

# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)

# get a URL for the status of the job
returned_sweep_job.services["Studio"].endpoint

# Download best trial model output
#ml_client.jobs.download(returned_sweep_job.name, output_name="model")



FileNotFoundError: [Errno 2] No such file or directory: '/data/6DIMCOCO/../../blobs/a2a88b96561196777ca173b15309ea859f4d2ce0'

In [10]:
from azure.ai.ml import command
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

inputs={

       "input_data": Input(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/",
                        mode=InputOutputModes.DIRECT),
}
registered_model_name = "buildimagenet"
job = command(
    code=os.path.join(project_dir),  # location of source code
    command="python BuildImagenet.py --data_path ${{inputs.input_data}}",
    environment=pipeline_job_env.name+"@latest",
    compute="smander",
    experiment_name="downloadmagenet",
    display_name="imgnetdownload",
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,
    },
    inputs=inputs,
)
ml_client.create_or_update(job)

Uploading 6DIMCOCO (84.87 MBs): 100%|██████████| 84869030/84869030 [00:01<00:00, 59438882.90it/s]




Experiment,Name,Type,Status,Details Page
downloadmagenet,cool_shelf_gnl95gtgmz,command,Starting,Link to Azure Machine Learning studio


In [9]:
import subprocess
from azure.ai.ml import command, Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes

# Define the source and destination paths
source_path = "../."
destination_path = "azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/"
# # Run the azcopy command to copy the directory
# subprocess.run(["./azcopy", "copy", source_path, destination_path, "--recursive"])

# Define the input and output paths
inputs = {
    "source_path": Input(type=AssetTypes.URI_FOLDER,
                         path=source_path,
                         mode=InputOutputModes.RO_MOUNT),
}

outputs = {
    "destination_path": Output(type=AssetTypes.URI_FOLDER,
                             path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data2/",
                               mode=InputOutputModes.RW_MOUNT),


}

# Create your command
job = command(
    command="cp -r ${{inputs.source_path}} ${{outputs.destination_path}}",
    compute="smander",
    environment=pipeline_job_env.name+"@latest",
    experiment_name="AzCopyJob",
    display_name="AzCopy",
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,
    },
    inputs=inputs,
    outputs=outputs,
)

# Submit the job
ml_client.create_or_update(job)


Exception: 
[37m
[30m
1) One or more fields are invalid[39m[39m

Details: 

[31m(x) Supported input path value are ARM id, AzureML id, remote uri or local path.
Met <class 'UnicodeEncodeError'>:
'utf-8' codec can't encode character '\udcc7' in position 1: surrogates not allowed[39m

Resolutions: 
1) Double-check that all specified parameters are of the correct types and formats prescribed by the Job schema.
If using the CLI, you can also check the full log in debug mode for more details by adding --debug to the end of your command

Additional Resources: The easiest way to author a yaml specification file is using IntelliSense and auto-completion Azure ML VS code extension provides: [36mhttps://code.visualstudio.com/docs/datascience/azure-machine-learning.[39m To set up VS Code, visit [36mhttps://docs.microsoft.com/azure/machine-learning/how-to-setup-vs-code[39m


In [7]:
#Lets define some sweeps, We really want to trial a load of values for all the following:
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy
from azure.identity import DefaultAzureCredential

# Create your command
command_job_for_sweep = command(
    code=".",  # location of source code
    command="python launch.py --num_trials 0 --dir ${{inputs.input_data}}",#--data ${{inputs.datadir}}",
    environment=pipeline_job_env.name+"@latest",
    compute="cpu-cluster",
    experiment_name="6DIMCOCOSWEEP2",
    display_name="6DIMSweepStephenMd1",
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,
    },
    inputs={
        "prune":Choice([1,0]),
        "projection":Choice(["None","inv","iinv"]),
        "normlogits":Choice([1,0]),
        "exactlabels":Choice([1,0]),
        "meanloss":Choice([1,0]),
        "maskLosses":Choice([0,1,2]),
        "logitsversion":Choice([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]),
        "precision":Choice([32,16]),
        "codeversion":Choice([6]),
        "transformer_layers":Choice([3,4,5,6,8,12]),
        "transformer_heads":Choice([16]),
        "embed_dim":Choice([64,128,512]),
        "transformer_width":Choice([64,128,512]),
        "devices":Choice([1]),
        "accelerator":Choice(["gpu"]),
        "log_path":os.path.join(project_dir,"./logs"),
        "batch_size":Choice(list(range(2,8))),
        "dims":Choice([3,6])
    },
)

# Call sweep() on your command job to sweep over your parameter expressions
sweep_job = command_job_for_sweep.sweep(
    compute="sparc-v100-low-priority", 
    sampling_algorithm="random",
    primary_metric="train_loss",#should really set this to something at the validation stage 
    goal="Minimize",
)
# Define the limits for this sweep
sweep_job.set_limits(max_total_trials=500, max_concurrent_trials=20, timeout=14400)

# Set early stopping on this one
sweep_job.early_termination = MedianStoppingPolicy(delay_evaluation=5, evaluation_interval=2)

# Specify your experiment details
sweep_job.display_name = "CLIP-SWEEP2"
sweep_job.experiment_name = "StephenM-CLIP-HighDimSweep"
sweep_job.description = "Run a hyperparameter sweep 6D repo"

# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)

# get a URL for the status of the job
returned_sweep_job.services["Studio"].endpoint

# Download best trial model output
#ml_client.jobs.download(returned_sweep_job.name, output_name="model")



FileNotFoundError: [Errno 2] No such file or directory: '/data/6DIMCOCO/../../blobs/a2a88b96561196777ca173b15309ea859f4d2ce0'

In [10]:
#Create job to delete : the file at  azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/annotations

outputToEdit={
    "remove":Output(type=AssetTypes.URI_FILE,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/coco/",
                        mode=InputOutputModes.RW_MOUNT
                        ),    
}

job = command(
    command="rm -rf ${{outputs.remove}}",#--data ${{inputs.datadir}}",
    environment=pipeline_job_env.name+"@latest",
    compute="sparc-v100-low-priority",
    experiment_name="datasetcleaning",
    display_name="temp",
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,
    },
    outputs=outputToEdit,
)
ml_client.create_or_update(job)

Experiment,Name,Type,Status,Details Page
datasetcleaning,strong_room_fv0nkdk93y,command,Starting,Link to Azure Machine Learning studio
